# Prueba de evaluación usando sympy

---

## Funciones del tipo checksol()

In [1]:
import sympy as sy

In [2]:
x = sy.symbols('x', real=True)
f = sy.Function('f', real=True)(x)

Construir una EDO

In [3]:
ode = sy.Eq(f.diff(x, 2) - 4 * f, 2 * sy.exp(2 * x) + 5 * sy.exp(- 2 * x))
display(ode)

Eq(-4*f(x) + Derivative(f(x), (x, 2)), 2*exp(2*x) + 5*exp(-2*x))

Resolviendo por método de coeficientes indeterminados

In [4]:
ode_ah = sy.Eq(ode.lhs, 0)
display(ode_ah)

Eq(-4*f(x) + Derivative(f(x), (x, 2)), 0)

In [5]:
D = sy.Symbol('D')
edo_op = ode_ah.subs(f.diff(x,2), D**2 * f)
display(edo_op)

Eq(D**2*f(x) - 4*f(x), 0)

In [6]:
m = sy.Symbol('m')
ec_char = edo_op.subs({D: m, f:1})
display(ec_char)

Eq(m**2 - 4, 0)

In [7]:
roots = sy.roots(ec_char, m)
display(roots)

{-2: 1, 2: 1}

In [8]:
a, b = [sy.exp(root * x) if times == 1 else [x**i * sy.exp(root * x) for i in range(times)] for root, times in roots.items()]
display(a, b)

exp(-2*x)

exp(2*x)

In [30]:
C1, C2 = sy.symbols('C(1:3)')
sol_ah = sy.Eq(f, C1 * a + C2 * b)
display(sol_ah)

Eq(f(x), C1*exp(-2*x) + C2*exp(2*x))

In [31]:
ec_char2 = sy.Eq(ec_char.lhs.factor() * ec_char.lhs, ec_char.rhs)
display(ec_char2)

Eq((m - 2)*(m + 2)*(m**2 - 4), 0)

In [32]:
roots2 = sy.roots(ec_char2, m)
display(roots2)

{2: 2, -2: 2}

In [33]:
(b, d), (a, c) = [sy.exp(root * x) if times == 1 else [x**i * sy.exp(root * x) for i in range(times)] for root, times in roots2.items()]
display(a, b, c, d)

exp(-2*x)

exp(2*x)

x*exp(-2*x)

x*exp(2*x)

In [34]:
C3, C4 = sy.symbols('C(3:5)')
sol_anh = sy.Eq(f, C1 * a + C2 * b + C3 * c + C4 * d)
display(sol_anh)

Eq(f(x), C1*exp(-2*x) + C2*exp(2*x) + C3*x*exp(-2*x) + C4*x*exp(2*x))

In [35]:
A, B = sy.symbols('A B')
sol_pq = sy.Eq(f, (sol_anh.rhs - sol_ah.rhs).subs({C3:A, C4:B}))
display(sol_pq)

Eq(f(x), A*x*exp(-2*x) + B*x*exp(2*x))

In [36]:
pars = ode.subs(f, sol_pq.rhs).simplify().expand()
display(pars)

Eq(2*exp(2*x) + 5*exp(-2*x), -4*A*exp(-2*x) + 4*B*exp(2*x))

In [37]:
parA, parB = [sy.solve(sy.Eq(pars.lhs.coeff(sy.exp(root * x), 1), pars.rhs.coeff(sy.exp(root * x), 1)), par)[0] for root, par in zip(roots, [A, B])]
display(parA, parB)

-5/4

1/2

Solución del alumno

In [38]:
gensol_nh = sy.Eq(f, sol_ah.rhs + sol_pq.rhs.subs({A:parA, B:parB}))
display(gensol_nh)

Eq(f(x), C1*exp(-2*x) + C2*exp(2*x) + x*exp(2*x)/2 - 5*x*exp(-2*x)/4)

Solución de SymPy

In [39]:
sy.dsolve(ode, f)

Eq(f(x), (C1 - 5*x/4)*exp(-2*x) + (C2 + x/2)*exp(2*x))

Built-in function de SymPy para evaluar expresiones

In [40]:
sy.checkodesol(ode, gensol_nh)
# ode: ecuación diferencial original
# gensol_nh: solución obtenida por el alumno

(True, 0)

## ¿Cómo evaluar la respuesta del alumno?

In [20]:
import sympy as sy

In [21]:
x, y = sy.symbols('x y')
f = sy.Function('f')(x)
u = sy.Function('u')(x, y)

In [22]:
equation = sy.Eq(2 * x**2, 4 + x)
ode = sy.Eq(f.diff(x, 2) + 7 * f.diff(x) + 10 * f, sy.exp(3 * x))
pde = sy.Eq(x * u.diff(x) - y * u.diff(y) + x**2 * u, y**2)

In [23]:
display(equation, ode, pde)

Eq(2*x**2, x + 4)

Eq(10*f(x) + 7*Derivative(f(x), x) + Derivative(f(x), (x, 2)), exp(3*x))

Eq(x**2*u(x, y) + x*Derivative(u(x, y), x) - y*Derivative(u(x, y), y), y**2)

Verificar si la expresión contiene una derivada

In [24]:
def whatis(expr):
    if isinstance(expr, sy.Basic): # verificar que la expresión sea una expresión de SymPy
        if expr.atoms(sy.Derivative): # verificar si la expresión contiene una derivada
            return True # es una ODE/PDE
        else:
            return False # es una expresión/ecuación
    else:
        return False # es un sistema de ecuaciones

Si la expresión contiene la clase ```Derivative```, estamos ante una ODE/PDE. Si no, es una simple ecuación lineal/no lineal

Se define una función que permita corroborar el resultado obtenido por el alumno.

Esta consiste de estructuras ```try/except``` e ```if/else``` anidadas, a manera de poder revisar con detenimiento todos los casos presentes.

In [25]:
def verificar_sympy(problema, solucion):
    # Caso para ODEs/PDEs
    if whatis(problema):
        # Caso para ODEs
        try:
            # Se intenta clasificar la ODE (PDEs no pasan esta prueba), por lo
            # que se realiza esta prueba primero. Además, las soluciones de PDEs
            # pasan la prueba de checkodesol, pero con resultado False
            sy.classify_ode(problema)
            # Se verifica la solución de la ODE. En caso de existir algún error con
            # la entrada, entrará el except del bloque anidado, dado que si el 
            # problema fuese una PDE, la línea anterior habría entrado al except 
            # de este bloque
            if not sy.checkodesol(problema, solucion)[0]:
                return print('Revisa tu solución')
            else:
                return print('¡Excelente! Tu solución es correcta')
        except Exception:
            # Caso para PDEs
            try:
                # Se intenta clasificar la PDE (ODEs no pasan esta prueba)
                # Contrario al caso anterior, las soluciones de ODEs no pasan
                # la prueba de checkpdesol.
                sy.classify_pde(problema)
                # Se intenta verificar la solución de la PDE. En caso de existir
                # algún error en la entrada, entrará el except
                if not sy.checkpdesol(problema, solucion)[0]:
                    return print('Revisa tu solución')
                else:
                    return print('¡Excelente! Tu solución es correcta')
            except Exception:
                # La entrada presenta un error, se debe revisar por el usuario
                print('Ha ocurrido un error. Revisa tu entrada')
    # Caso para ecuaciones/sistemas de ecuaciones
    else:
        try:
            # Para pasar el bloque try, debe cumplirse el siguiente caso:
            # - El problema depende de sólo n variables, es un problema lineal y
            #   existe sólo una solución
            # Esto dado a que la verificación asume que la solución que recibe es
            # un diccionario. En caso de que la solución sea una lista de diccionarios
            # entrará en el bloque except
            if sy.checksol(problema, solucion):
                return print('¡Excelente! Tu solución es correcta')
            else:
                return print('Revisa tu solución')
        except Exception:
            # Para entrar al bloque except, debe presentarse el siguiente caso:
            # - El problema depende de n variables, pero no es un problema lineal por lo
            #   que existe más de una solución
            # Por ende, se revisa primero que la solución sea una lista (los diccionarios
            # tienen len(dict) = 1). En dicho caso, se almacena el resultado de todas las
            # soluciones en una lista y después se verifica que todos los elementos de la
            # lista tengan valor True.
            if len(solucion) > 1:
                resultado = [sy.checksol(problema, sol) for sol in solucion]
                if all(resultado):
                    return print('¡Excelente! Tu solución es correcta')
                else:
                    return print('Revisa tu solución')
            else:
                # Si se entra a este caso, existe un error en la entrada y esta debe ser
                # revisada por el usuario
                return print('Ha ocurrido un error. Revisa tu entrada')

Verificando la solución de la ODE (generada por sympy)

In [26]:
ode_sol = sy.dsolve(ode, f)
display(ode_sol)

Eq(f(x), C1*exp(-5*x) + C2*exp(-2*x) + exp(3*x)/40)

In [27]:
verificar_sympy(ode, ode_sol)

¡Excelente! Tu solución es correcta


Verificando la solución de la PDE (generada por sympy)

In [28]:
# La PDE utilizada para este ejemplo es una PDE de primer orden
# dado que pdesolve sólo puede resolver este tipo de ecuaciones
pde_sol = sy.pdsolve(pde)
display(pde_sol)

Eq(u(x, y), (y**2*(x**2*Ei(x**2/2) - 2*exp(x**2/2))/4 + F(x*y))*exp(-x**2/2))

In [29]:
verificar_sympy(pde, pde_sol)

¡Excelente! Tu solución es correcta


Verificar la solución de una ecuación no lineal de una sola variable

In [30]:
eq_sol = sy.solve(equation, dict=True)
display(*eq_sol)

{x: 1/4 - sqrt(33)/4}

{x: 1/4 + sqrt(33)/4}

In [31]:
verificar_sympy(equation, eq_sol)

¡Excelente! Tu solución es correcta


Verificar la solución de una ecuación lineal de dos variables

In [32]:
z = sy.Symbol('z')
equation2 = sy.Eq(x + y, 2*z), sy.Eq(y, -4*z)
display(*equation2)

Eq(x + y, 2*z)

Eq(y, -4*z)

In [33]:
eq2_sol = sy.solve(equation2)
display(eq2_sol)

{x: 6*z, y: -4*z}

In [34]:
verificar_sympy(equation2, eq2_sol)

¡Excelente! Tu solución es correcta


Verificar la solución de una ecuación no lineal de dos variables

In [35]:
equation3 = sy.Eq(x**2 + y**2, 2*z), sy.Eq(y**2, -4*z)
display(*equation3)

Eq(x**2 + y**2, 2*z)

Eq(y**2, -4*z)

In [36]:
eq3_sol = sy.solve(equation3)
display(*eq3_sol)

{x: -sqrt(6)*sqrt(z), y: -2*sqrt(-z)}

{x: -sqrt(6)*sqrt(z), y: 2*sqrt(-z)}

{x: sqrt(6)*sqrt(z), y: -2*sqrt(-z)}

{x: sqrt(6)*sqrt(z), y: 2*sqrt(-z)}

In [37]:
verificar_sympy(equation3, eq3_sol)

¡Excelente! Tu solución es correcta
